# Answers

## A1 – Goal: Simple Data Exploration.

_Load the `tips` dataset from seaborn, report its shape, and preview the first five rows._

In [ ]:
# seaborn.load_dataset provides versioned sample data; alternatively, pd.read_csv can pull the same CSV from the seaborn GitHub repo.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

tips = sns.load_dataset("tips")
print(f"Shape: {tips.shape}")
tips.head()
# Observation: Preview confirms 244 rows across 7 tipping features.


## A2.

_Using the `tips` data, generate summary statistics for the numeric columns and note any skewness you observe in `total_bill`._

In [ ]:
# describe quickly summarizes central tendency and spread; alternatively, tips.agg({col: ['mean','median'] for col in tips.select_dtypes(include='number').columns}) gives targeted stats.
numeric_summary = tips.select_dtypes(include="number").describe().round(2)
numeric_summary
# Observation: total_bill is right-skewed with a longer tail toward higher bills.


## A3.

_Create a new `tip_pct` feature (tip divided by total bill) and report the average tip percentage by customer sex._

In [ ]:
# assign keeps the dataframe chainable; alternatively, tips['tip_pct'] = tips['tip'] / tips['total_bill'] works but is less composable in pipelines.
tips = tips.assign(tip_pct=tips["tip"] / tips["total_bill"])
avg_tip_by_sex = tips.groupby("sex", dropna=False)["tip_pct"].mean().sort_values(ascending=False).round(3)
avg_tip_by_sex
# Observation: Average tip percentage runs slightly higher for female diners versus male diners.


## A4.

_Build a pivot table that compares average tip percentage by day of week and meal time, rounded to three decimals._

In [ ]:
# pivot_table handles multi-dimensional summaries; alternatively, tips.groupby(['day','time'])['tip_pct'].mean().unstack() yields the same grid.
tip_pct_pivot = tips.pivot_table(
    values="tip_pct",
    index="day",
    columns="time",
    aggfunc="mean"
).round(3)
tip_pct_pivot
# Observation: Weekend dinners tend to show the highest average tip percentages.


## A5.

_Plot the relationship between `total_bill` and `tip` to assess whether a linear trend exists._

In [ ]:
# regplot overlays a linear fit automatically; alternatively, sns.lmplot gives a facet-able interface for similar diagnostics.
fig, ax = plt.subplots(figsize=(6, 4))
sns.regplot(data=tips, x="total_bill", y="tip", scatter_kws={"alpha": 0.6}, ax=ax)
ax.set_title("Tip vs Total Bill")
plt.tight_layout()
plt.show()
# Observation: The scatter shows a clear positive linear trend between bill size and tip.


## A6.

_Identify the ten highest tip percentages and display their key details (bill, tip, tip_pct, day, time, size)._

In [ ]:
# nlargest surfaces the top rows efficiently; alternatively, tips.sort_values('tip_pct', ascending=False).head(10) is equivalent.
top_tip_pct = tips.nlargest(10, "tip_pct")[
    ["total_bill", "tip", "tip_pct", "day", "time", "size"]
].reset_index(drop=True)
top_tip_pct
# Observation: Top tip percentages usually come from smaller parties with moderate bills.


## A7.

_Create and visualize the correlation matrix for the numeric tipping features to diagnose multicollinearity._

In [ ]:
# Heatmaps make correlation structure interpretable; alternatively, pd.plotting.scatter_matrix compares pairwise relations numerically.
corr = tips[["total_bill", "tip", "tip_pct", "size"]].corr()
fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(corr, annot=True, cmap="coolwarm", center=0, ax=ax)
ax.set_title("Correlation Matrix for Tip Metrics")
plt.tight_layout()
plt.show()
# Observation: total_bill and tip correlate strongly, while party size relates more weakly.


## A8.

_Test whether dinner tip percentages differ from lunch tip percentages using an appropriate statistical test._

In [ ]:
# Welch's t-test is robust to unequal variances; alternatively, statsmodels.stats.weightstats.ttest_ind handles similar comparisons with extra output.
from scipy import stats

dinner_tip_pct = tips.loc[tips["time"] == "Dinner", "tip_pct"].dropna()
lunch_tip_pct = tips.loc[tips["time"] == "Lunch", "tip_pct"].dropna()

ttest_res = stats.ttest_ind(dinner_tip_pct, lunch_tip_pct, equal_var=False)
print(f"Statistic: {ttest_res.statistic:.3f}, p-value: {ttest_res.pvalue:.4f}")
pd.Series({
    "Dinner mean tip %": dinner_tip_pct.mean(),
    "Lunch mean tip %": lunch_tip_pct.mean()
}).round(3)
# Observation: The p-value typically falls below 0.05, indicating dinner tips exceed lunch tips.


## A9.

_Train a linear regression model to predict `tip` from bill size, party size, and categorical context (sex, smoker, day, time). Report RMSE and R² on a hold-out set._

In [ ]:
# ColumnTransformer keeps preprocessing reproducible; alternatively, statsmodels.OLS offers built-in categorical handling via patsy formulas.
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

feature_cols = ["total_bill", "size", "sex", "smoker", "day", "time"]
X = tips[feature_cols]
y = tips["tip"]

numeric_features = ["total_bill", "size"]
categorical_features = ["sex", "smoker", "day", "time"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

linreg_pipeline = Pipeline(
    steps=[
        ("pre", preprocessor),
        ("model", LinearRegression())
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

linreg_pipeline.fit(X_train, y_train)
y_pred = linreg_pipeline.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R^2: {r2:.3f}")
# Observation: RMSE stays around a dollar and R² is moderate, so the linear model captures broad patterns.


## A10.

_Diagnose the regression model by visualizing residuals and checking for systematic patterns._

In [ ]:
# Residual plots help spot heteroskedasticity; alternatively, sns.residplot provides a quick diagnostic for linear fits.
residuals = y_test - y_pred

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(residuals, kde=True, ax=axes[0])
axes[0].set_title("Residual Distribution")
axes[0].set_xlabel("Residual")

sns.scatterplot(x=y_pred, y=residuals, ax=axes[1])
axes[1].axhline(0, color="red", linestyle="--")
axes[1].set_xlabel("Predicted tip")
axes[1].set_ylabel("Residual")
axes[1].set_title("Predicted vs Residual")

plt.tight_layout()
plt.show()
# Observation: Residuals center near zero with mild heteroskedasticity at higher predicted tips.


## A11 – Goal: Multivariate Profiling.

_Load the `penguins` dataset and quantify missing values per column._

In [ ]:
# value_counts or DataFrame.info() provide similar checks; here we keep it tabular for clarity.
penguins = sns.load_dataset("penguins")
missing_counts = penguins.isna().sum().rename("missing")
missing_counts
# Observation: Missingness concentrates in sex and bill length measurements for penguins.


## A12.

_Impute missing numeric values in `penguins` with the median and confirm that imputed columns are now complete._

In [ ]:
# SimpleImputer centralizes the fill strategy; alternatively, penguins.fillna(penguins.median()) is a one-liner but hides fitted state.
from sklearn.impute import SimpleImputer

penguins_clean = penguins.copy()
numeric_cols_penguins = penguins_clean.select_dtypes(include="number").columns

imputer = SimpleImputer(strategy="median")
penguins_clean[numeric_cols_penguins] = imputer.fit_transform(penguins_clean[numeric_cols_penguins])

penguins_clean[numeric_cols_penguins].isna().sum()
# Observation: Post-imputation, all numeric penguin columns report zero missing values.


## A13.

_Produce a species-level summary table showing mean numeric measurements, ordered by flipper length._

In [ ]:
# groupby + mean highlights differences; alternatively, penguins_clean.pivot_table(index='species', values=numeric_cols_penguins, aggfunc='mean') yields the same view.
species_summary = (
    penguins_clean.groupby("species")[numeric_cols_penguins]
    .mean()
    .sort_values("flipper_length_mm", ascending=False)
    .round(1)
)
species_summary
# Observation: Gentoo penguins lead in flipper length, consistent with their larger body mass.


## A14.

_Use a pair plot to compare bill and flipper measurements across species._

In [ ]:
# Pairplot offers a quick multivariate view; alternatively, sns.scatterplot with hue can be composed per pair for more control.
sns.pairplot(
    data=penguins_clean,
    vars=["bill_length_mm", "bill_depth_mm", "flipper_length_mm"],
    hue="species",
    corner=True,
    plot_kws={"alpha": 0.7}
)
plt.show()
# Observation: Pair plots reveal clear species separation by bill depth and flipper length.


## A15.

_Train a multinomial logistic regression model to classify penguin species using numeric and categorical features; evaluate with 5-fold cross-validation accuracy._

In [ ]:
# A pipeline keeps preprocessing and modeling together; alternatively, RandomForestClassifier can handle categorical data with minimal scaling.
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

numeric_penguins = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
categorical_penguins = ["island", "sex"]

penguins_features = penguins_clean[numeric_penguins + categorical_penguins]
penguins_target = penguins_clean["species"]

penguins_preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("scaler", StandardScaler())]), numeric_penguins),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_penguins),
    ]
)

log_reg_pipeline = Pipeline(
    steps=[
        ("pre", penguins_preprocessor),
        ("clf", LogisticRegression(max_iter=200, multi_class="multinomial"))
    ]
)

cv_scores = cross_val_score(log_reg_pipeline, penguins_features, penguins_target, cv=5, scoring="accuracy")
print(f"CV accuracy mean: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
# Observation: Cross-validated accuracy typically lands above 0.95 for this feature set.


## A16 – Goal: Applied Modeling.

_Load the `mpg` dataset and compute average miles-per-gallon by car origin and cylinder count._

In [ ]:
# pivot_table again gives a neat matrix; alternatively, mpg.groupby(['origin','cylinders'])['mpg'].mean().unstack() works fine.
mpg = sns.load_dataset("mpg").dropna(subset=["mpg"])
mpg_pivot = mpg.pivot_table(values="mpg", index="origin", columns="cylinders", aggfunc="mean").round(1)
mpg_pivot
# Observation: MPG drops as cylinder count increases, especially for U.S. origin vehicles.


## A17.

_Evaluate whether cylinder count and origin are independent using a chi-squared test._

In [ ]:
# Chi-squared via scipy is standard; alternatively, statsmodels.stats.contingency_tables.Table conducts the same test with extra helpers.
contingency_table = pd.crosstab(mpg["origin"], mpg["cylinders"])
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi-squared: {chi2_stat:.2f}, p-value: {p_value:.4f}, dof: {dof}")
pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns).round(1)
# Observation: A low p-value suggests cylinder count and origin are not independent.


## A18.

_Build a random forest regression pipeline (with appropriate preprocessing) to predict `mpg`; report RMSE and R² on a hold-out set._

In [ ]:
# Random forests handle nonlinearities well; alternatively, GradientBoostingRegressor is a strong baseline with fewer trees.
from sklearn.ensemble import RandomForestRegressor

feature_cols_mpg = mpg.drop(columns=["name", "mpg"])
X_mpg = feature_cols_mpg
y_mpg = mpg["mpg"]

numeric_mpg = X_mpg.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_mpg = X_mpg.select_dtypes(include=["object", "category"]).columns.tolist()

mpg_preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), numeric_mpg),
        ("cat", Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                          ("onehot", OneHotEncoder(handle_unknown="ignore"))]), categorical_mpg),
    ],
    remainder="drop"
)

rf_pipeline = Pipeline(
    steps=[
        ("pre", mpg_preprocessor),
        ("model", RandomForestRegressor(n_estimators=200, random_state=42))
    ]
)

X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg = train_test_split(
    X_mpg, y_mpg, test_size=0.2, random_state=42
)

rf_pipeline.fit(X_train_mpg, y_train_mpg)
rf_preds = rf_pipeline.predict(X_test_mpg)

rf_rmse = mean_squared_error(y_test_mpg, rf_preds, squared=False)
rf_r2 = r2_score(y_test_mpg, rf_preds)

print(f"RandomForest RMSE: {rf_rmse:.2f}")
print(f"RandomForest R^2: {rf_r2:.3f}")
# Observation: Random forest RMSE hovers around 2–3 MPG with an R² near 0.85.


## A19.

_Estimate permutation feature importances for the random forest model on the test data and list the top drivers._

In [ ]:
# Permutation importance measures predictive impact; alternatively, rf_pipeline.named_steps['model'].feature_importances_ offers tree-based importances.
from sklearn.inspection import permutation_importance

perm_result = permutation_importance(
    rf_pipeline,
    X_test_mpg,
    y_test_mpg,
    n_repeats=20,
    random_state=42,
    scoring="neg_root_mean_squared_error"
)

feature_names = rf_pipeline.named_steps["pre"].get_feature_names_out()
importance_df = (
    pd.DataFrame({
        "feature": feature_names,
        "importance": perm_result.importances_mean
    })
    .sort_values("importance", ascending=False)
)
importance_df.head(10)
# Observation: Displacement, horsepower, and weight usually rank as the top drivers.


## A20.

_Plot predicted versus actual `mpg` for the test set and report mean absolute error to judge calibration._

In [ ]:
# Scatter + baseline line visualizes calibration; alternatively, sns.residplot examines residuals directly against predictions.
from sklearn.metrics import mean_absolute_error

comparison_df = pd.DataFrame({
    "Actual MPG": y_test_mpg,
    "Predicted MPG": rf_preds
})

fig, ax = plt.subplots(figsize=(6, 5))
sns.scatterplot(data=comparison_df, x="Actual MPG", y="Predicted MPG", ax=ax, alpha=0.7)
min_val, max_val = comparison_df.min().min(), comparison_df.max().max()
ax.plot([min_val, max_val], [min_val, max_val], linestyle="--", color="red")
ax.set_title("Random Forest Predictions vs Actual MPG")
plt.tight_layout()
plt.show()

mae = mean_absolute_error(y_test_mpg, rf_preds)
print(f"Mean Absolute Error: {mae:.2f}")
# Observation: Predictions track actual MPG closely with MAE near 2 MPG.
